In [1]:
import requests
import json
import numpy as np
from pyquil import Program

def get_t_value(QPU=2, qubit=0, T=1):

    url = "https://forest-server.qcs.rigetti.com/lattices/Aspen-M-"

    if T not in [1, 2]:
        raise ValueError("t must be 1 or 2")
    
    if QPU not in [2, 3]:
        raise ValueError("QPU must be 2 or 3")

    response = requests.get(url + str(QPU))

    file = json.loads(response.text)

    qs = file["lattice"]["specs"]["1Q"]

    if str(qubit) not in qs:
        raise ValueError("Qubit not found")

    return qs[str(qubit)]["T" + str(T)]

def damping_channel(damp_prob=.1):
    """
    Generate the Kraus operators corresponding to an amplitude damping
    noise channel.

    :params float damp_prob: The one-step damping probability.
    :return: A list [k1, k2] of the Kraus operators that parametrize the map.
    :rtype: list
    """
    damping_op = np.sqrt(damp_prob) * np.array([[0, 1],
                                                [0, 0]])

    residual_kraus = np.diag([1, np.sqrt(1-damp_prob)])
    return [residual_kraus, damping_op]

def append_kraus_to_gate(kraus_ops, g):
    """
    Follow a gate `g` by a Kraus map described by `kraus_ops`.

    :param list kraus_ops: The Kraus operators.
    :param numpy.ndarray g: The unitary gate.
    :return: A list of transformed Kraus operators.
    """
    return [kj.dot(g) for kj in kraus_ops]


def append_damping_to_gate(gate, damp_prob=.1):
    """
    Generate the Kraus operators corresponding to a given unitary
    single qubit gate followed by an amplitude damping noise channel.

    :params np.ndarray|list gate: The 2x2 unitary gate matrix.
    :params float damp_prob: The one-step damping probability.
    :return: A list [k1, k2] of the Kraus operators that parametrize the map.
    :rtype: list
    """
    return append_kraus_to_gate(damping_channel(damp_prob), gate)


p = Program()
p.define_noisy_gate






<bound method Program.define_noisy_gate of <pyquil.quil.Program object at 0x000002779F5F5050>>